In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Load the data

In [2]:
import datetime
from helpers import *
from costs import *
from plots import *
from gradient_descent import *
from stochastic_gradient_descent import *

x_train, features = load_csv_data("data/dataset/x_train.csv")
y_train, _ = load_csv_data("data/dataset/y_train.csv")
x_test, _ = load_csv_data("data/dataset/x_test.csv")

In [3]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))

(328135, 321)
(328135,)
(109379, 321)


In [4]:
# Show the first 5 rows of the data and statistics
data = pd.DataFrame(x_train, columns=features)

data.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,53.0,11.0,11162015.0,11.0,16.0,2015.0,1100.0,2.015016e+09,2.015016e+09,NaN,...,1.0,1.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,2.0
1,33.0,12.0,12152015.0,12.0,15.0,2015.0,1200.0,2.015004e+09,2.015004e+09,1.0,...,9.0,9.0,3.0,3.0,4.0,9.0,9.0,NaN,NaN,NaN
2,20.0,10.0,10202015.0,10.0,20.0,2015.0,1100.0,2.015006e+09,2.015006e+09,1.0,...,4.0,2.0,2.0,2.0,3.0,1.0,1.0,1.0,2.0,2.0
3,42.0,6.0,6182015.0,6.0,18.0,2015.0,1100.0,2.015005e+09,2.015005e+09,NaN,...,2.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,2.0
4,24.0,11.0,11062015.0,11.0,6.0,2015.0,1100.0,2.015005e+09,2.015005e+09,1.0,...,9.0,9.0,3.0,3.0,4.0,1.0,1.0,9.0,9.0,2.0


In [5]:
data.describe()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
count,328135.000000,328135.000000,3.281350e+05,328135.000000,328135.000000,328135.000000,328135.000000,3.281350e+05,3.281350e+05,188720.0,...,328135.000000,328135.000000,326252.000000,326252.000000,326252.000000,328135.000000,328135.000000,116757.00000,116757.000000,296055.000000
mean,29.973651,6.360690,6.563638e+06,6.416661,14.496235,2015.024758,1115.014857,2.015005e+09,2.015005e+09,1.0,...,3.488567,2.737611,2.717093,2.814279,3.652698,1.820421,1.882390,2.28991,2.406794,1.966675
std,16.031777,3.487263,3.489248e+06,3.492739,8.334495,0.155387,35.721753,4.118475e+03,4.118475e+03,0.0,...,2.395363,2.445086,1.317887,1.348067,1.259353,2.355332,2.346068,2.51678,2.771851,1.434133
min,1.000000,1.000000,1.012016e+06,1.000000,1.000000,2015.000000,1100.000000,2.015000e+09,2.015000e+09,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
25%,19.000000,3.000000,3.232015e+06,3.000000,8.000000,2015.000000,1100.000000,2.015002e+09,2.015002e+09,1.0,...,2.000000,2.000000,2.000000,2.000000,3.000000,1.000000,1.000000,1.00000,1.000000,1.000000
50%,29.000000,6.000000,6.242015e+06,6.000000,14.000000,2015.000000,1100.000000,2.015004e+09,2.015004e+09,1.0,...,3.000000,2.000000,3.000000,3.000000,4.000000,1.000000,1.000000,1.00000,1.000000,2.000000
75%,44.000000,9.000000,1.002202e+07,10.000000,21.000000,2015.000000,1100.000000,2.015007e+09,2.015007e+09,1.0,...,4.000000,2.000000,3.000000,3.000000,4.000000,1.000000,1.000000,2.00000,2.000000,2.000000
max,72.000000,12.000000,1.231202e+07,12.000000,31.000000,2016.000000,1200.000000,2.015023e+09,2.015023e+09,1.0,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000
